# Vector search
* https://github.com/DataTalksClub/llm-zoomcamp/tree/main/03-vector-search
* https://www.youtube.com/watch?v=ptByfB_YcEg&list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&index=19

In [ ]:
# Run ElasticSearch Docker image
# https://youtu.be/ptByfB_YcEg?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=42
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [9]:
!pip install sentence_transformers==2.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 835.1 kB/s eta 0:00:0031m20.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:008 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 9.7 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 58.3 MB/s eta 0:00:000m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 43.5 MB/s eta 0:00:000m eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
# Load Python libraries
import requests

## Prepare documents
https://youtu.be/ptByfB_YcEg?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=319

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()

In [6]:
docs_raw

[{'course': 'data-engineering-zoomcamp',
  'documents': [{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
    'section': 'General course-related questions',
    'question': 'Course - When will the course start?'},
   {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
    'section': 'General course-related questions',
    'question': 'Course - What are the prerequisites for this course?'},
   {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in

In [5]:
# https://youtu.be/ptByfB_YcEg?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=356
documents = []

for course in docs_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
documents[0:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines 

## Create embeddings using pretrained models
https://youtu.be/ptByfB_YcEg?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=400

In [7]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Pretrained models - https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
# Will load model from https://huggingface.co/sentence-transformers/all-mpnet-base-v2
model = SentenceTransformer("all-mpnet-base-v2")

In [16]:
encoded_txt = model.encode("This is a simple sentence")
encoded_txt

array([ 4.44875564e-03, -7.61314258e-02, -3.77468328e-04,  7.52523402e-03,
       -3.80979776e-02,  3.80131453e-02, -9.73008294e-03, -5.05397702e-03,
       -9.37976502e-03,  1.23887584e-02,  4.91276123e-02,  1.52210230e-02,
        3.80008705e-02, -6.41802400e-02,  9.42127407e-03, -5.19749001e-02,
        9.08066332e-02,  1.71115622e-02,  1.62125528e-02,  2.98865885e-02,
        1.50541600e-03,  8.35078582e-03,  3.78841944e-02, -1.01192761e-02,
        6.46108761e-03,  3.97424155e-05, -1.45217031e-02, -1.88468415e-02,
       -3.74039710e-02, -1.51667662e-03, -1.02680055e-02, -3.68062854e-02,
        2.36677658e-02, -6.46023452e-02,  1.96967039e-06, -5.01107657e-03,
       -2.80828192e-03, -1.92073956e-02, -8.65119696e-02,  2.83465385e-02,
       -5.38667664e-02,  3.63705941e-02, -2.26468481e-02,  2.87367962e-02,
       -1.32342121e-02,  1.08689629e-01,  3.70518453e-02,  3.38802189e-02,
       -5.30679226e-02,  3.61782461e-02, -1.35725585e-03, -3.63483503e-02,
       -2.78346464e-02, -

In [18]:
# Length of the embedded text
# Length could differ from model to model
len(encoded_txt)

768

In [9]:
# Create the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transform "Title" into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [21]:
operations[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.03570346161723137,
  -0.06891428679227829,
  -0.04448342323303223,
  0.006250153295695782,
  -0.042247094213962555,
  -0.00645399559289217,
  0.02754814364016056,
  -0.01861027255654335,
  -0.05203750357031822,
  -0.0024047393817454576,
  0.05745450034737587,
  -0.015592600218951702,
  0.015488838776946068,
  0.0987016037106514,
  0.01564955711364746,
  -0.061

## Setup ElasticSearch connection
https://youtu.be/ptByfB_YcEg?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=795

In [10]:
# Initiate ElasticSearch connection
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '30234f656e35', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'uxa_cUBmSU6Por8O3eKRvQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Create mappings and index
https://youtu.be/ptByfB_YcEg?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=829

In [13]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

In [14]:
# Create indices
index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## Add documents into index
https://youtu.be/ptByfB_YcEg?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=1073

In [15]:
# Index dataset
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

## Create end-user query
https://youtu.be/ptByfB_YcEg?list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&t=1177

In [16]:
search_term = "windows or mac?"

In [17]:
vector_search_term = model.encode(search_term)

In [18]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000,
}

In [31]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])

In [33]:
res['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'zjfWtpABVrhJP3UtMry3',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': '4TfWtpABVrhJP3Utob-U',
  '_score': 0.61347336,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

## Perform keyword search with semantic search
https://www.youtube.com/watch?v=ptByfB_YcEg&list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&index=19

In [20]:
# Keyword search
search_term = "windows or python?"
course_filter = "data-engineering-zoomcamp"
response = es_client.search(index=index_name,
                            query = {
                                "bool": {
                                    "must": {
                                        "multi_match":
                                                      {"query": search_term,
                                                       "fields": ["text", "question", "course", "title"],
                                                       "type": "best_fields"
                                                       }
                                             },
                                         "filter": {
                                             "term": {"course": course_filter}}
                                    }})

In [21]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'qLuBt5ABvvdnb85u0rG2',
  '_score': 7.728908,
  '_source': {'text': 'Problem: If you have already installed pgcli but bash doesn\'t recognize pgcli\nOn Git bash: bash: pgcli: command not found\nOn Windows Terminal: pgcli: The term \'pgcli\' is not recognized…\nSolution: Try adding a Python path C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts to Windows PATH\nFor details:\nGet the location: pip list -v\nCopy C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\site-packages\n3. Replace site-packages with Scripts: C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts\nIt can also be that you have Python installed elsewhere.\nFor me it was under c:\\python310\\lib\\site-packages\nSo I had to add c:\\python310\\lib\\Scripts to PATH, as shown below.\nPut the above path in "Path" (or "PATH") in System Variables\nReference: https://stackoverflow.com/a/68233660',
   'section': 'Module 1: Docker and Terraform',
   'question': 'PGCLI - pg

In [22]:
search_term

'windows or python?'

In [23]:
# Semantic search
vector_search_term = model.encode(search_term)
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000,
}

In [24]:
response = es_client.search(index=index_name,
                            query={"match":{"course": course_filter}},
                            knn=knn_query,
                            size=5)
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'UruBt5ABvvdnb85ux7HC',
  '_score': 1.4823202,
  '_source': {'text': 'Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]\nBut Python 3.10 and 3.11 should work fine.',
   'section': 'General course-related questions',
   'question': 'Environment - Is Python 3.9 still the recommended version to use in 2024?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.008795942179858685,
    0.01882270537316799,
    0.008203202858567238,
    -0.024633260443806648,
    0.029965609312057495,
    0.020212220028042793,
    0.11235181987285614,
    0.008089477196335793,
    0.006306793075054884,
    -0.020710287615656853,
    0.009332780726253986,
    0.01337830163538456,
    -0.026092836633324623,
    0.05249353498220444,
    -0.017481058835983276,
    -0.06065770611166954,
    0.024950070306658745,
    -0.0065099624916911125,
    0.043373409658670425,
    0.0436139777302742,
    -0.02519589476287365,
    

In [25]:
# explain=True -> shows how the scores are calculated
response = es_client.search(index=index_name,
                            query={"match":{"course": course_filter}},
                            knn=knn_query,
                            size=5,
                            explain=True)
response['hits']['hits']

[{'_shard': '[course-questions][0]',
  '_node': 'SQtQJuhnS8Gc9IuzEZ5mDQ',
  '_index': 'course-questions',
  '_id': 'UruBt5ABvvdnb85ux7HC',
  '_score': 1.4823202,
  '_source': {'text': 'Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]\nBut Python 3.10 and 3.11 should work fine.',
   'section': 'General course-related questions',
   'question': 'Environment - Is Python 3.9 still the recommended version to use in 2024?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.008795942179858685,
    0.01882270537316799,
    0.008203202858567238,
    -0.024633260443806648,
    0.029965609312057495,
    0.020212220028042793,
    0.11235181987285614,
    0.008089477196335793,
    0.006306793075054884,
    -0.020710287615656853,
    0.009332780726253986,
    0.01337830163538456,
    -0.026092836633324623,
    0.05249353498220444,
    -0.017481058835983276,
    -0.06065770611166954,
    0.024950070306658745,
    -0.0065099624916911125,
    0.